In [1]:
%reset
import matplotlib.pyplot as plt
import rasterio 
from rasterio.plot import plotting_extent, show
from rasterio.merge import merge
import geopandas as gpd
from osgeo import gdal
import numpy as np
from skimage import io
import sys

ortho_NIR = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/nir.tif"
ortho_rgb = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/rgb.tif"
ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/mspec.tif"

g = gdal.Open(ortho_NIR)

b3 = g.GetRasterBand(2).ReadAsArray().astype(np.float32)
b4 = g.GetRasterBand(3).ReadAsArray().astype(np.float32)
nir_ndvi = (b4 - b3)/(b4 + b3)

c = gdal.Open(ortho_mspec)
c3 = c.GetRasterBand(2).ReadAsArray().astype(np.float32)
c4 = c.GetRasterBand(4).ReadAsArray().astype(np.float32)
mspec_ndvi = (c4 - c3)/(c4 + c3)


dataset = rasterio.open(ortho_mspec)
x = gdal.Open(ortho_mspec)
x.SetGeoTransform(g.GetGeoTransform())
x.SetProjection(g.GetProjection())
print('Multispectral bands :', dataset.count)
mspec_green = dataset.read(1)
print(mspec_green.shape)
mspec_red = dataset.read(2)
mspec_rededge = dataset.read(3)
mspec_nir = dataset.read(4)
mspec_bands = [mspec_green, mspec_red, mspec_rededge, mspec_nir, mspec_ndvi]


dataset = rasterio.open(ortho_NIR)
x = gdal.Open(ortho_NIR)
x.SetGeoTransform(g.GetGeoTransform())
x.SetProjection(g.GetProjection())
print('NIR bands :', dataset.count)
nir_green = dataset.read(1)
print(nir_green.shape)
nir_red = dataset.read(2)
nir_nir = dataset.read(3)
nir_bands = [nir_green, nir_red, nir_nir, nir_ndvi]

dataset = rasterio.open(ortho_rgb)
x = gdal.Open(ortho_rgb)
x.SetGeoTransform(g.GetGeoTransform())
x.SetProjection(g.GetProjection())
print('rgb bands :', dataset.count)
rgb_red = x.GetRasterBand(1).ReadAsArray().astype(np.float32)
print(rgb_red.shape)
rgb_green = x.GetRasterBand(2).ReadAsArray().astype(np.float32)
rgb_blue =  x.GetRasterBand(3).ReadAsArray().astype(np.float32)


C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_12396/3195988833.py:20: RuntimeWarning: invalid value encountered in true_divide
  nir_ndvi = (b4 - b3)/(b4 + b3)
C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_12396/3195988833.py:25: RuntimeWarning: invalid value encountered in true_divide
  mspec_ndvi = (c4 - c3)/(c4 + c3)


Multispectral bands : 5
(2135, 2362)
NIR bands : 4
(5675, 6279)
rgb bands : 4
(6442, 7126)


In [3]:
dsReprj_mspec= gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_mspec.tif", c, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
dsReprj_nir = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_nir.tif", g, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])

stack = []
for i in range(1,4):
    stack.append(x.GetRasterBand(i).ReadAsArray().astype(np.float32))

for i in range(1,6):
    stack.append(dsReprj_mspec.GetRasterBand(i).ReadAsArray().astype(np.float32))

for i in range(1,5):
    stack.append(dsReprj_nir.GetRasterBand(i).ReadAsArray().astype(np.float32))


stack_data = np.array(stack, dtype=object)
driver = gdal.GetDriverByName ( "GTiff" )
DataSet = driver.Create("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/output_file_channelFusion.tif", stack_data[1].shape[1], stack_data[1].shape[0], len(stack_data), gdal.GDT_Float32)
DataSet.SetGeoTransform(x.GetGeoTransform())
DataSet.SetProjection(x.GetProjection())
print('---------------------')
for i, image in enumerate(stack_data, 1):
    DataSet.GetRasterBand(i).WriteArray(image)
    DataSet.GetRasterBand(i).SetNoDataValue(-9999.0)
DataSet.FlushCache()

---------------------


In [4]:
print(type(stack))
for i, image in enumerate(stack_data, 1):
    print(np.min(image), ",", np.max(image))
    print(image.shape[0], ",", image.shape[1])


<class 'list'>
0.0 , 254.0
6442 , 7126
0.0 , 254.0
6442 , 7126
0.0 , 255.0
6442 , 7126
0.0 , 254.0
6442 , 7126
0.0 , 255.0
6442 , 7126
0.0 , 248.0
6442 , 7126
0.0 , 243.0
6442 , 7126
0.0 , 255.0
6442 , 7126
0.0 , 255.0
6440 , 7125
0.0 , 202.0
6440 , 7125
0.0 , 205.0
6440 , 7125
0.0 , 255.0
6440 , 7125


In [5]:
NIR = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/output_file_nir.tif"
RGB = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/output_file_rgb.tif"
MSPEC = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/output_file_mspec.tif"

driverTiff = gdal.GetDriverByName('GTiff')
NIR = gdal.Open(NIR)
RGB = gdal.Open(RGB)
MSPEC = gdal.Open(MSPEC)

nbands = NIR.RasterCount
band_data = []
print('bands', NIR.RasterCount, 'rows', NIR.RasterYSize, 'columns', NIR.RasterXSize)
for i in range(1, nbands+1):
        band = NIR.GetRasterBand(i).ReadAsArray()
        band_data.append(band)

band_data = np.dstack(band_data)
print(band_data.shape)

nbands = RGB.RasterCount
band_data = []
print('bands', RGB.RasterCount, 'rows', RGB.RasterYSize, 'columns', RGB.RasterXSize)
for i in range(1, nbands+1):
        band = RGB.GetRasterBand(i).ReadAsArray()
        band_data.append(band)

band_data = np.dstack(band_data)
print(band_data.shape)


nbands = MSPEC.RasterCount
band_data = []
print('bands', MSPEC.RasterCount, 'rows', MSPEC.RasterYSize, 'columns', MSPEC.RasterXSize)
for i in range(1, nbands+1):
        band = MSPEC.GetRasterBand(i).ReadAsArray()
        band_data.append(band)

band_data = np.dstack(band_data)
print(band_data.shape)

bands 4 rows 5675 columns 6279
(5675, 6279, 4)
bands 3 rows 6442 columns 7126
(6442, 7126, 3)
bands 5 rows 2135 columns 2362
(2135, 2362, 5)
